In [1]:
import pygmo as pg  # See https://esa.github.io/pygmo2/install.html for installation (conda instructions)

import numpy as np
import pandas as pd
from time import time

from airsenal.framework.utils import (
    NEXT_GAMEWEEK,
    CURRENT_SEASON,
    list_players,
    get_latest_prediction_tag,
    get_predicted_points_for_player,
)
from airsenal.framework.team import Team, TOTAL_PER_POSITION

In [2]:
gw_start = NEXT_GAMEWEEK
num_gw = 3
gw_range = list(range(gw_start, min(38, num_gw)))

tag = get_latest_prediction_tag()

In [3]:
# PyGMO User Defined Problem
class OptTeam:
    def __init__(
        self,
        gw_range,
        tag,
        budget=1000,
        season=CURRENT_SEASON,
        bench_boost_gw=None,
        triple_captain_gw=None,
        remove_zero=True,  # don't consider players with predicted pts of zero
    ):
        self.season = season
        self.gw_range = gw_range
        self.start_gw = min(gw_range)
        self.bench_boost_gw = bench_boost_gw
        self.triple_captain_gw = triple_captain_gw

        self.tag = tag
        self.positions = ["GK", "DEF", "MID", "FWD"]
        self.n_squad_players = sum(TOTAL_PER_POSITION.values())
        self.budget = budget
        
        self.players, self.position_idx = self._get_player_list()
        if remove_zero:
            self._remove_zero_pts()
        self.n_available_players = len(self.players)

          
    def fitness(self, player_ids):
        """
        PyGMO required function.
        The objective function to minimise. And constraints to evaluate.
        """
        # Make team from player IDs
        team = Team(budget=self.budget)
        for idx in player_ids:
            team.add_player(
                self.players[int(idx)].player_id,
                season=self.season,
                gameweek=self.start_gw,
            )
            
        # Check team is valid
        if not team.is_complete():
            return [0]
        
        # Calc expected points for all gameweeks
        score = 0.0
        for gw in self.gw_range:
            if gw == self.bench_boost_gw:
                score += team.get_expected_points(gw, self.tag, bench_boost=True)
            elif gw == self.triple_captain_gw:
                score += team.get_expected_points(gw, self.tag, triple_captain=True)
            else:
                score += team.get_expected_points(gw, self.tag)
        
        return [-score]
    
    def get_bounds(self):
        """
        PyGMO required function.
        Defines min and max value for each parameter.
        """
        # use previously calculated position index ranges to set the bounds
        # to force all attempted solutions to contain the correct number of
        # players for each position.
        low_bounds = []
        high_bounds = []
        for pos in self.positions:
            low_bounds += [self.position_idx[pos][0]] * TOTAL_PER_POSITION[pos]
            high_bounds += [self.position_idx[pos][1]] * TOTAL_PER_POSITION[pos]
        
        return (low_bounds, high_bounds)
    
    def get_nec(self):
        """PyGMO function.
        Defines number of equality constraints."""
        return 0
    
    def get_nix(self):
        """
        PyGMO function.
        Number of integer dimensions.
        """
        return self.n_squad_players
    
    def gradient(self, x):
        return pg.estimate_gradient_h(lambda x: self.fitness(x), x)
    
    def _get_player_list(self):
        """
        Get list of active players at the start of the gameweek range,
        and the id range of players for each position.
        """
        players = []
        change_idx = [0]
        # build players list by position (e.g. all GK, then all DEF etc.)
        for pos in self.positions:
            players += list_players(position=pos,
                                    season=self.season,
                                    gameweek=self.start_gw)
            change_idx.append(len(players))
        
        # min and max idx of players for each position
        position_idx = {self.positions[i-1]: (change_idx[i-1], change_idx[i]-1) 
                        for i in range(1, len(change_idx))}
        return players, position_idx
    
    def _remove_zero_pts(self):
        players = []
        change_idx = [0]
        last_pos = self.positions[0]
        for p in self.players:
            gw_pts = get_predicted_points_for_player(p, self.tag, season=self.season)
            total_pts = sum([pts for gw, pts in gw_pts.items() if gw in self.gw_range])
            if total_pts > 0:
                if p.position(self.season) != last_pos:
                    change_idx.append(len(players))
                    last_pos = p.position(self.season)
                players.append(p)
        change_idx.append(len(players))
        
        position_idx = {self.positions[i-1]: (change_idx[i-1], change_idx[i]-1) 
                        for i in range(1, len(change_idx))}
                
        self.players = players
        self.position_idx = position_idx
    


In [4]:
opt_team = OptTeam(
    gw_range,
    tag,    
)


In [5]:
# Build problem
opt_team = OptTeam(
    gw_range,
    tag,
)

prob = pg.problem(opt_team)

print(prob)

Problem name: <class '__main__.OptTeam'>
	Global dimension:			15
	Integer dimension:			15
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [0, 0, 27, 27, 27, ... ]
	Upper bounds: [26, 26, 147, 147, 147, ... ]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 15
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0
	Gradient evaluations: 0

	Thread safety: none



In [6]:
# Create algorithm to solve problem with
algo = pg.algorithm(uda = pg.sga(gen=100))
#algo = pg.algorithm(uda = pg.gaco(gen=2))
#algo = pg.algorithm(uda = pg.ihs(gen=1000))
algo.set_verbosity(1)
print(algo)

# population of problems
pop = pg.population(prob=prob, size=100)


Algorithm name: SGA: Genetic Algorithm [stochastic]
	Thread safety: basic

Extra info:
	Number of generations: 100
	Crossover:
		Type: exponential
		Probability: 0.9
	Mutation:
		Type: polynomial
		Probability: 0.02
		Distribution index: 1
	Selection:
		Type: tournament
		Tournament size: 2
	Seed: 1102824068
	Verbosity: 1


In [7]:
# solve problem
pop = algo.evolve(pop)

print("Best score:", -pop.champion_f[0], "pts")

Best score: 102.92169558966603 pts


In [11]:
team = Team(budget=opt_team.budget)

for idx in pop.champion_x:
    print(opt_team.players[int(idx)].position(CURRENT_SEASON),
          opt_team.players[int(idx)].name,
          opt_team.players[int(idx)].team(CURRENT_SEASON, 1),
          opt_team.players[int(idx)].price(CURRENT_SEASON, 1))
    team.add_player(
        opt_team.players[int(idx)].player_id,
        season=opt_team.season,
        gameweek=opt_team.start_gw,
    )
print(f"£{team.budget/10}m in the bank")
print("="*10)
pts = team.get_expected_points(opt_team.start_gw, tag)
print(f"GW{opt_team.start_gw}: {pts:.0f} pts")
print(team)

GK Jordan Pickford EVE 50
GK Alisson Ramses Becker LIV 60
DEF Trent Alexander-Arnold LIV 75
DEF Tyrone Mings AVL 50
DEF Joel Ward CRY 45
DEF Serge Aurier TOT 55
DEF Luke Thomas LEI 45
MID Wilfried Zaha CRY 70
MID Steven Alzate BHA 45
MID Pierre-Emerick Aubameyang ARS 120
MID James Maddison LEI 70
MID Heung-Min Son TOT 90
FWD Raúl Jiménez WOL 85
FWD Richarlison de Andrade EVE 80
FWD Jordan Ayew CRY 60
£0.0m in the bank
GW1: 53 pts

=== starting 11 ===


== GK ==

Alisson Ramses Becker (LIV)

== DEF ==

Trent Alexander-Arnold (LIV)(VC)
Joel Ward (CRY)
Serge Aurier (TOT)
Luke Thomas (LEI)

== MID ==

Wilfried Zaha (CRY)
Pierre-Emerick Aubameyang (ARS)(C)
James Maddison (LEI)
Heung-Min Son (TOT)

== FWD ==

Raúl Jiménez (WOL)
Jordan Ayew (CRY)

=== subs ===

Richarlison de Andrade (EVE)
Jordan Pickford (EVE)
Steven Alzate (BHA)
Tyrone Mings (AVL)



In [9]:
print(opt_team.position_idx)
print(pop.champion_x)

{'GK': (0, 26), 'DEF': (27, 147), 'MID': (148, 296), 'FWD': (297, 343)}
[ 11.   0.  27.  60. 115.  52. 119. 166. 284. 148. 168. 155. 305. 306.
 320.]


In [10]:
print(team)


=== starting 11 ===


== GK ==

Alisson Ramses Becker (LIV)

== DEF ==

Trent Alexander-Arnold (LIV)(VC)
Joel Ward (CRY)
Serge Aurier (TOT)
Luke Thomas (LEI)

== MID ==

Wilfried Zaha (CRY)
Pierre-Emerick Aubameyang (ARS)(C)
James Maddison (LEI)
Heung-Min Son (TOT)

== FWD ==

Raúl Jiménez (WOL)
Jordan Ayew (CRY)

=== subs ===

Richarlison de Andrade (EVE)
Jordan Pickford (EVE)
Steven Alzate (BHA)
Tyrone Mings (AVL)

